# Предсказание цен

![](data/math01_01.png)

Давайте немного пофантазируем и даже помечтаем: вы успешно закончили магистратуру, нашли первую работу на позиции начинающего дата-сайентиста, а затем начался стремительный карьерный рост и рост вашего благосостояния. В итоге сложился первый капитал. Во что его эффективно вложить? Присмотримся к рынку недвижимости и возьмем на вооружение метод **OLS**.

> В этом проекте мы проанализируем зависимость цен на недвижимость от множества факторов, а нашей задачей будет научиться предсказывать стоимость по имеющимся данным.

Задачи в проекте будут соответствующими: 
- вначале мы поразмыслим об эффективности вычислений;
- затем подумаем, как применить элементы математической статистики для решении этой задачи;
- напоследок закрепим одну из базовых операций — получение **обратной матрицы**. 

Все вышеперечисленное понадобится вам как для выполнения проекта, так и для работы с реальными данными.

Итак, начнём: попробуем построить модель, которая предскажет стоимость квартиры в городе Ъ по некоторому набору входных данных.

Исходные данные отражены в таблице:

In [5]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/math01_data.csv')
df

,Комнаты,Площадь,Этаж,Центр?,Спальный1?,Спальный 2?,Цена
0,3,51,3,0,1,0,2200
1,1,30,1,0,1,0,1600
2,2,45,2,0,1,0,1900
3,3,55,1,0,1,0,2000
4,1,45,3,1,0,0,4500
5,3,100,3,1,0,0,7000
6,2,71,2,1,0,0,5000
7,1,31,2,0,0,1,1700
8,3,53,5,0,0,1,2100
9,1,33,3,0,0,1,1500


В таблице для каждой квартиры указана стоимость, количество комнат, площадь и этаж. Столбцы `Центр?`, `Спальный1?` и `Спальный2?` говорят о районе, в котором расположена квартира. Обратите внимание, что три последних признака бинарные, то есть в соответствующем признаке лежит значение 1, если квартира находится в данном районе, и 0, если в другом.

Наша цель будет состоять в том, чтобы построить **линейную модель**. Это значит, что для набора признаков _X_ мы хотим подобрать такие коэффициенты _A_, чтобы `sum(A[i]*X[i])` было как можно ближе к реальной цене.

## Задание 1. Разминка. Умножение матриц

При работе с большими данными необходимо учитывать **оптимальность выполнения расчётов**.

Из теории к этому модулю вы уже познакомились с умножением матриц. Но знали ли вы, что эта операция ассоциативна? Иными словами, для любых трёх матриц _A_, _B_, _C_ произведение _A * B * C_ будет одинаковым независимо от порядка выполнения умножений (не перепутайте порядок аргументов _A * B_ — не всегда равно _B * A_).

### Поясним на примере

- _(A * B) * C_ — мы можем сначала выполнить умножение _A * B_, а потом результат умножить на _C_.
- _A * (B * C)_ — или сначала умножить _B * C_, а потом результат умножить на _A_.

В обоих случаях результат умножения будет одинаковый: _(A * B) * C == A * (B * C)_. Но одинакова ли скорость этих операций?

Напомним, что матрицы _A_ и _B_ можно умножить в том случае, если количество столбцов в матрице _A_ совпадает с количеством строк в матрице _B_. В терминах `numpy` это означает, что `A.shape[1] == B.shape[0]`. Само умножение происходит так: элемент `[i, j]` результирующей матрицы равен сумме произведений `sum(A[i, k]*B[k, j])`.

### Задание

Для выполнения этого задания определите, в каком порядке эффективнее умножить три матрицы. Например, у нас есть матрицы `A = [[1, 2]]`, `B = [[2], [1]]`, `C = [[5]]`. Эффективнее сначала умножить _A * B_ (два умножения), а потом умножить результат на _C_ (одно умножение). В этом случае мы выполним три умножения.

Если же мы будем умножать сначала _B * C_ (два умножения), а потом результат умножим слева на _A_ (два умножения), то в сумме будет четыре умножения.

**Входные данные:** матрицы _A, B, C_ в виде _numpy_-массивов. Гарантируется, что их можно перемножить в порядке _A * B * C_.

**Результат:** напишите функцию `multiplication_order(A, B, C)`, которая вернёт строку `"(AxB)xC"`, если количество умножений элементов матриц при умножении _(A * B) * C_ меньше либо равно количеству умножений, если выполнять их в порядке _A * (B * C)_. В противном случае верните строку `"Ax(BxC)"`.

- Пример входных данных: `A = [[1, 2]]`, `B=[[2], [1]]`, `C=[[5]]`
- Пример результата: `(AxB)xC`. 

In [12]:
def multiplication_order(A, B, C):
    ab_shape = (A.shape[0], B.shape[1])
    ab_complexity = A.shape[0] * B.shape[1] * A.shape[1] + ab_shape[0] * C.shape[1] * ab_shape[1]

    bc_shape = (B.shape[0], C.shape[1])
    bc_complexity = A.shape[0] * bc_shape[1] * A.shape[1] + B.shape[0] * C.shape[1] * B.shape[1]

    if ab_complexity <= bc_complexity:
        return '(AxB)xC'
    return 'Ax(BxC)'

In [13]:
a = np.array([[1, 2]])
b = np.array([[2], [1]])
c = np.array([[5]])

multiplication_order(a, b, c)

'(AxB)xC'

## Задание 2. Полезность признаков

А теперь применим магию линейной алгебры к решению задачи **Feature Engineering** (этап разработки признаков). Более подробно о _feature engineering_ мы поговорим на пятой неделе курса _Математика и алгоритмы для машинного обучения в модуле ML-2. Предобработка данных_. Кстати, вы уже сталкивались с признаками, когда решали проект **Титаник** в рамках курса _Программирование на Python_.

Признаки должны давать модели всю информацию о моделируемом процессе — поэтому очень важно найти такие признаки, которые будут сильно влиять на результат.

Вернёмся к теме проекта: предсказанию цен на недвижимость. Мы усердно поработали и теперь пришло время вложить солидный капитал, заработанный при помощи анализа данных, в недвижимость. Для начала нужно понять, какие признаки наиболее и наименее влияют на стоимость квартир.

С другой стороны, если признаков слишком много, могут возникнуть следующие проблемы: **переобучение** и **неэффективность**. Под переобучением мы понимаем настолько хорошее «запоминание» тренировочного множества, что на тестовом множестве результат оказывается плохим. Другими словами, модель «запомнила» тренировочные данные вместо того, чтобы найти в них закономерности.

### Задача

В этой задаче вам потребуется найти признак, наиболее сильно коррелирующий с результатом, и найти самый «слабый» признак.

**Входные данные:** матрица признаков _X_ в виде _numpy_-массива и вектор цен _Y_. Количество строк в матрице _X_ совпадает с количеством элементов в векторе _Y_. Каждая строка матрицы описывает признаки одной квартиры, а соответствующий элемент в _Y_ равен цене квартиры.

**Результат:** напишите функцию `best_worst(X, Y)`, которая вернёт два числа: `max_corr_idx` (номер признака, наиболее коррелирующего с ценой) и `min_corr_idx` (номер признака, наименее коррелирующего с ценой). Учитывайте, что корреляция имеет знак, а сила корреляции зависит от абсолютного значения — нужно вернуть наибольший и наименьший признаки по абсолютному значению. Подсказка: можно использовать функцию `numpy.corrcoef` для быстрого вычисления коэффициентов корреляции.

- Пример входных данных: см. таблицу из предисловия к проекту.
- Пример результата: функция должна вернуть кортеж `(3, 2)`. `3` — номер (начиная с 0) столбца `Центр?`, цена больше всего зависит от того, находится ли квартира в центре. `2` — номер столбца `Этаж`. В этом примере от номера этажа цена практически не зависит.

In [50]:
def best_worst(X, Y):
    n = X.shape[1] # number of columns in X
    coeffs = []
    for i in range(n):
        x = X[:, i]
        m = np.corrcoef(x, Y)
        corr = m[0, 1]
        coeffs.append((i, np.abs(corr)))
    coeffs = sorted(coeffs, key=lambda x: x[1])
    
    min_corr_idx = coeffs[0][0]
    max_corr_idx = coeffs[-1][0]
    
    return max_corr_idx, min_corr_idx
    

In [48]:
# Решение от автора:
def best_worst(x, y):
    c = np.abs(np.corrcoef(np.concatenate([y.reshape((y.shape[0], 1)), x], axis=1).T)[0][1:])
    return np.argmax(c), np.argmin(c)

In [51]:
x = df.drop('Цена', axis=1).values
y = df['Цена'].values

best_worst(x, y)

(3, 2)

## Задание 3. Зависимость признаков

В предыдущей задаче мы исследовали связь признаков с результатом: стоимостью недвижимости. А можно ли что-то сказать о связях между самими признаками? Для этого в данной задаче вам потребуется найти ранг матрицы корреляции.

Применительно к нашей задаче: предсказанию цен на недвижимость поиск зависимых между собой признаков может быть полезен для оптимизации модели. Напомним, что если ранг матрицы корреляции меньше её размерности, то какие-то из признаков, которые мы рассматриваем, могут быть **линейно зависимы** от других, а значит могут быть проигнорированы. Если же все рассматриваемые признаки обладают отдельной значимостью, то ранг матрицы корреляции будет равен её размерности.

**Входные данные:** матрица признаков в _X_ виде _numpy_-массива.

**Результат:** напишите функцию `corr_rank(X)`, возвращающую одно число — ранг корреляционной матрицы. Подсказка: можно использовать функцию `np.linalg.matrix_rank`.

- Пример входных данных: см. таблицу из предисловия к проекту.
- Пример результата: 5

In [112]:
def corr_rank(X):
    c = np.corrcoef(X.T)
    return np.linalg.matrix_rank(c)

In [113]:
x = df.drop('Цена', axis=1).values
corr_rank(x)

5

## Задание 4. Нахождение обратной матрицы

Начнём процесс построения модели, которая выберет оптимальные варианты для инвестиций в недвижимость: модель сможет предсказывать цены на квартиры по набору признаков. Для этого мы воспользуемся знакомым нам методом _OLS_. Сперва нужно найти обратную матрицу. В этой задаче по входной квадратной матрице _A_ нужно определить, возможно ли найти обратную матрицу, и, если возможно, вернуть её.

**Входные данные:** матрица _A_ в виде _numpy_-массива

**Результат:** напишите функцию `inverse_matrix(A)`, которая вернёт `None`, если матрица необратима (то есть её определитель по абсолютному значению меньше 0.001), либо вернёт обратную матрицу в виде _numpy_-массива. Подсказка: используйте `np.linalg.inv`

- Пример входных данных: `A = np.array([[1, 2], [2, 1]])`
- Пример результата: `array([[-0.33333333, 0.66666667], [ 0.66666667, -0.33333333]])`

In [74]:
np.linalg.det(A)

-2.9999999999999996

In [78]:
def inverse_matrix(A):
    if A.shape[0] != A.shape[1]:
        return None
    d = np.linalg.det(A)
    if np.abs(d) < 0.001:
        return None
    return np.linalg.inv(A)

In [79]:
A = np.array([[1, 2], [2, 1]])
inverse_matrix(A)

array([[-0.33333333,  0.66666667],
       [ 0.66666667, -0.33333333]])

## Задание 5. Построение модели

Обратная матрица найдена. Теперь мы готовы найти философский камень — построить линейную модель методом _OLS_! Из теории вам уже известна формула `a = np.linalg.inv(X.T * X) * X.T * y`, осталось её только запрограммировать.

### Пояснение

Матрица `Q = np.linalg.inv(X.T * X) * X.T` — это обобщение обратной матрицы для случая матриц, не являющихся квадратными. Действительно, если матрица не квадратная, то для неё невозможно найти обратную в привычном смысле этого слова. Однако при выполнении некоторых условий нам подойдёт матрица _Q_ (кстати, она называется **матрицей Мура-Пенроуза**).

![](data/math01_02.png)

Давайте запрограммируем эту формулу!

**Входные данные:** матрица признаков _X_ в виде _numpy_-массива и вектор цен _y_ в виде _numpy_-массива. _X_ имеет форму `(m, n)` — _m_ строк, по строке на каждую квартиру, _n_ признаков для каждой квартиры. _y_ имеет форму `(m)`, это вектор _m_ из элементов — цены всех _m_ квартир.

**Результат:** напишите функцию `fit_model(X, y)`, которая вернёт _numpy_-массив с оптимальными коэффициентами _a_, найденными методом _OLS_.

- Пример входных данных: см. таблицу из предисловия к проекту.
- Пример результата: `[-574.12295766 65.33255763 141.80223878 1566.16246224 12.32450391 -315.34552489]`

In [103]:
def fit_model(X, y):
    a = np.linalg.inv(X.T @ X) @ X.T @ y
    return a

In [105]:
x = df.drop('Цена', axis=1).values
y = df['Цена'].values

fit_model(x, y)

array([-574.12295766,   65.33255763,  141.80223878, 1566.16246224,
         12.32450391, -315.34552489])